In [207]:
pip install scikit-learn nltk 


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [208]:
import pandas as pd
import yaml
from sqlalchemy import create_engine
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords

## Database Connection

In [209]:
with open('config.yml', 'r') as f: #Abrir el archivo en modo de  lectura
    config = yaml.safe_load(f) # Crear un diccionario con lo que hay en el archivo
    config_db_etl = config['bodega'] #Obtener solo la configuración de la bodega
    config_db = config["fuente"] #Obtener solo la configuración de la bodega

In [210]:
# Construct the database URL
url_db_etl = (f"{config_db_etl['driver']}://{config_db_etl['user']}:{config_db_etl['password']}@{config_db_etl['host']}:"
           f"{config_db_etl['port']}/{config_db_etl['db']}")
url_db = (f"{config_db['driver']}://{config_db['user']}:{config_db['password']}@{config_db['host']}:"
           f"{config_db['port']}/{config_db['db']}")

In [211]:
# Create the SQLAlchemy Engine
etl_conn = create_engine(url_db_etl)
olap_conn = create_engine(url_db)

## Extraction

In [212]:
descripcion_novedad = pd.read_sql_table('dim_descripcion_novedad', etl_conn) 
novedad_servicio = pd.read_sql_table("mensajeria_novedadesservicio",url_db)

In [213]:
novedad_servicio.head(5)

,id,fecha_novedad,tipo_novedad_id,descripcion,servicio_id,es_prueba,mensajero_id
0,4,2023-11-30 05:00:00+00:00,1,A,51,True,7
1,5,2023-11-30 05:00:00+00:00,1,Halo,51,True,7
2,6,2023-11-30 05:00:00+00:00,1,A,51,True,7
3,7,2023-11-30 05:00:00+00:00,1,B,51,True,7
4,8,2023-11-30 05:00:00+00:00,1,A,51,True,7


## Transformation

In [214]:
# Asegúrate de tener los recursos de NLTK descargados
nltk.download('stopwords')
stop_words = list(stopwords.words('spanish'))  # Convertir a lista

# Suponiendo que tu DataFrame se llama novedad_servicio
# y contiene una columna 'descripcion'
descripciones = novedad_servicio['descripcion'].dropna().tolist()

# Paso 1: Crear una matriz TF-IDF
vectorizer = TfidfVectorizer(stop_words=stop_words)
X = vectorizer.fit_transform(descripciones)

# Paso 2: Usar K-Means para agrupar las descripciones
num_clusters = 10  # Cambia esto según el número de grupos que desees
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X)

# Paso 3: Agregar los resultados al DataFrame original
novedad_servicio['grupo'] = kmeans.labels_


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nicol\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [215]:
novedad_servicio

,id,fecha_novedad,tipo_novedad_id,descripcion,servicio_id,es_prueba,mensajero_id,grupo
0,4,2023-11-30 05:00:00+00:00,1,A,51,True,7,6
1,5,2023-11-30 05:00:00+00:00,1,Halo,51,True,7,6
2,6,2023-11-30 05:00:00+00:00,1,A,51,True,7,6
3,7,2023-11-30 05:00:00+00:00,1,B,51,True,7,6
4,8,2023-11-30 05:00:00+00:00,1,A,51,True,7,6
...,...,...,...,...,...,...,...,...
5203,5246,2024-08-31 17:56:46.191229+00:00,1,"Facturaron el refrigerante equivocado, se hará...",28455,True,27,6
5204,5247,2024-08-31 18:21:14.634785+00:00,2,Edte drrvicio lo hace angelo,28464,True,25,6
5205,5248,2024-08-31 19:55:19.569591+00:00,2,Edte lo hace csrlos,28467,True,25,6
5206,5249,2024-08-31 19:55:51.942719+00:00,2,Este lohace csrlos,28466,True,25,6


In [216]:
hecho_novedad_servicio = novedad_servicio[["id", "descripcion", "grupo"]]
hecho_novedad_servicio = pd.merge(hecho_novedad_servicio, descripcion_novedad[["key_descripcion_novedad"]], left_on="grupo", right_on="key_descripcion_novedad", how="left")
hecho_novedad_servicio

,id,descripcion,grupo,key_descripcion_novedad
0,4,A,6,6
1,5,Halo,6,6
2,6,A,6,6
3,7,B,6,6
4,8,A,6,6
...,...,...,...,...
5203,5246,"Facturaron el refrigerante equivocado, se hará...",6,6
5204,5247,Edte drrvicio lo hace angelo,6,6
5205,5248,Edte lo hace csrlos,6,6
5206,5249,Este lohace csrlos,6,6


In [217]:
des_cat = hecho_novedad_servicio[hecho_novedad_servicio['key_descripcion_novedad'] == 9]
des_cat.head(5)

,id,descripcion,grupo,key_descripcion_novedad
867,869,Sigo en la fila para enviar encomienda,9,9
1588,1587,A la espera en E.P. Terminal.,9,9
1630,3223,Esperando en terminal,9,9
1631,3225,Esperando en terminal,9,9
1651,3265,A la espera en E.P Terminal.,9,9


In [218]:
descripcion_novedad

,key_descripcion_novedad,grupo,terminos_clave,descripcion
0,0,Grupo 0,"espera, sigo, entreguen, reciban, masivo",Pendiente de Recepción o Confirmación
1,1,Grupo 1,"compañero, toma, servicio, entrego, continúa",Entrega por Compañero o Reparto Directo
2,2,Grupo 2,"esperando, despacho, bodega, entrega, reciban",Pendiente en Bodega o Almacenamiento
3,3,Grupo 3,"enterado, pendiente, recoger, ok, salgo",Recogida Pendiente o Confirmación en Curso
4,4,Grupo 4,"servicio, camioneta, mal, programado, abandono",Problemas con el Servicio o Programación
5,5,Grupo 5,"repuesto, espera, deja, asesor, señor",Espera de Repuestos o Asistencia Técnica
6,6,Grupo 6,"informa, entrega, encuentra, cerrado, b66",Entrega en Proceso o Lugar Cerrado
7,7,Grupo 7,"rod, latín, proveedor, aquí, toy",Pedidos con Proveedor o Ubicación en Espera
8,8,Grupo 8,"recibe, b1, asunto, trasbordo, queda",Recepción Completa o Transbordo
9,9,Grupo 9,"terminal, encomienda, enviar, alisten, espera",Esperando Envío desde Terminal


In [219]:
# Contar solicitudes por cada 'key_descripcion_novedad'
cantidad_por_descripcion_novedad = hecho_novedad_servicio.groupby("key_descripcion_novedad").size().reset_index(name="cantidad_por_descripcion_novedad")
# Hacer merge para agregar la columna 'cantidad_por_descripcion_novedad' al DataFrame original
hecho_novedad_servicio = hecho_novedad_servicio.merge(cantidad_por_descripcion_novedad, on="key_descripcion_novedad", how="left")
# Eliminar columnas innecesarias
hecho_novedad_servicio.drop(columns=["descripcion", "grupo"], inplace=True)
# Mostrar las primeras 10 filas del DataFrame resultante
hecho_novedad_servicio.head(10)

,id,key_descripcion_novedad,cantidad_por_descripcion_novedad
0,4,6,2207
1,5,6,2207
2,6,6,2207
3,7,6,2207
4,8,6,2207
5,9,6,2207
6,10,6,2207
7,11,6,2207
8,12,6,2207
9,15,6,2207


In [220]:
print(len(hecho_novedad_servicio))

5208


## Loading

In [221]:
hecho_novedad_servicio.to_sql("hecho_novedad_servicio", etl_conn, if_exists="replace", index_label="key_novedad_servicio") 

208